In [1]:
#DOCUMENTS_DIR = './documents'
VECTOR_DB_DIR = './vector_db'
#MODEL_DIR = './models'
NUM_RELEVANT_DOCS = 3
EMBEDDING_MODEL = 'paraphrase-MiniLM-L3-v2'
# LLM = 'distilbert/distilgpt2'
LLM = 'meta/llama-3.1-8b-instruct'
CLASSIFICATION_MODEL = "facebook/bart-large-mnli"

query = "I'm not able to play video smoothly"


In [2]:
import os
import sys
import time
from tqdm import tqdm
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma
from langchain_huggingface import HuggingFacePipeline
from langchain.schema import Document
import json
from langchain.prompts import PromptTemplate


In [3]:
# Loading API Keys
from dotenv import load_dotenv
load_dotenv()

True

In [4]:
def load_json(file_path):
    with open(file_path, 'r') as f:
        return json.load(f)

support_tickets = load_json('support_tickets.json')
knowledge_base = load_json('knowledge_base.json')

# docs = [Document(page_content=entry["content"]) for entry in knowledge_base]

In [5]:
docs = []

for ticket, entry in zip(support_tickets, knowledge_base):
    ticket_text = ticket['text']
    relevant_knowledge = entry['content']
    
    combined_content = f"Support Ticket: {ticket_text}\nRelevant Knowledge: {relevant_knowledge}"
    
    docs.append(combined_content)

In [35]:
# for doc in docs:
#     print(doc)

In [6]:
docs = [Document(page_content=combined_content) for combined_content in docs]

In [7]:
# def build(docs, vector_db_dir, embedding_model, embedding_model_name):
#     os.makedirs(vector_db_dir, exist_ok=True)
#     vector_db_path = os.path.join(vector_db_dir, embedding_model_name.split('/')[-1].split('.')[0])

#     if not os.path.exists(vector_db_path):
#         db = Chroma.from_documents(
#             documents=docs, 
#             embedding=embedding_model, 
#             persist_directory=vector_db_path
#         )
#         print(f'Vector Database Created at {vector_db_path}')
    
#     else:
#         db = Chroma(
#             persist_directory=vector_db_path,
#             embedding_function=embedding_model
#         )
#         print(f'Vector Database already exists at {vector_db_path}')

#     return db

In [7]:
embedding_model = HuggingFaceEmbeddings(model_name=EMBEDDING_MODEL)

/home/architgpt28/miniconda3/envs/EdLight_RAG/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
/home/architgpt28/miniconda3/envs/EdLight_RAG/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [8]:
os.makedirs(VECTOR_DB_DIR, exist_ok=True)
vector_db_path = os.path.join(VECTOR_DB_DIR, EMBEDDING_MODEL.split('/')[-1])

if not os.path.exists(vector_db_path):

    print(f'Creating Vector Database at {vector_db_path}')
    Chroma.from_documents(
        documents=docs, 
        embedding=embedding_model, 
        persist_directory=vector_db_path
    )

db = Chroma(
    persist_directory=vector_db_path,
    embedding_function=embedding_model
)

retriever = db.as_retriever(
    search_type="similarity",
    search_kwargs={"k":NUM_RELEVANT_DOCS}
)

In [9]:
retrieved_docs = retriever.get_relevant_documents(query)

for doc in retrieved_docs:
    print(doc.page_content)

/home/architgpt28/miniconda3/envs/EdLight_RAG/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


Support Ticket: The video playback is very laggy on my device.
Relevant Knowledge: Category 5 - Performance Issues - Performance issues can be related to device specifications, network connectivity, or app optimization.
Support Ticket: I can't find the option to change my profile picture.
Relevant Knowledge: Category 4 - Account Management - Account management includes tasks such as changing profile information, linking social media accounts, and managing privacy settings.
Support Ticket: The app crashes every time I try to upload a photo.
Relevant Knowledge: Category 2 - App Functionality - App crashes can be caused by outdated software or device incompatibility.


In [11]:
# if isinstance(QUERY, str):
#     relevant_docs = retriever.invoke(QUERY)

#     print('\nRelevant Documents:')
#     for i, doc in enumerate(relevant_docs):
#         print(f'Document {i}:\n\n{doc.page_content}\n')
#         if doc.metadata:
#             print(f"Source: {doc.metadata.get('Header 1', 'None')} -> {doc.metadata.get('Header 2', 'None')} -> {doc.metadata.get('Header 3', 'None')} -> {doc.metadata.get('Header 4', 'None')}\n")

#     times = []
#     for _ in tqdm(range(1000)):
#         start = time.time()
#         retriever.invoke(QUERY)
#         end = time.time()

#         times.append(end - start)

#     print(f'\nAverage Retreival Runtime: {(sum(times)/len(times))*1000} MilliSeconds')

#     sys.exit(0)

In [12]:
# llm = HuggingFacePipeline.from_model_id(
#     model_id=LLM,
#     task="text-generation",
#         pipeline_kwargs=dict(
#         max_new_tokens=10,
#         do_sample=False,
#         repetition_penalty=1.03,
#         temperature=0,
#     ),
#     model_kwargs=dict(
#         cache_dir="./cache"
#     ),
#     # trust_remote_code=True
# )

Loading checkpoint shards: 100%|██████████| 2/2 [00:14<00:00,  7.23s/it]


In [13]:


# # Define a prompt template for answering questions with context
# qa_system_prompt = (
#     "You are an assistant for question-answering tasks. Based on the following document, provide relevant knowledge to answer the query. If the document doesn't contain relevant information, say that you don't know. In the answer only give the relevant knowledge corresponding to the ticket.\n\nDocument:\n{document}\n\nQuery:\n{query}"
# )

# qa_prompt = PromptTemplate(
#     input_variables=["query", "document"],
#     template=qa_system_prompt
# )

In [17]:
# def retrieval_qa_chain(query):
#     # Retrieve relevant documents
#     docs = retriever.get_relevant_documents(query)
    
#     # Initialize the results list
#     answers = []

#     # Process each document with the QA chain
#     for doc in docs:
#         # Create a formatted prompt with the document and query
#         prompt = qa_prompt.format(query=query, document=doc.page_content)
#         # Get the answer from the LLM
#         answer = llm(prompt)
#         answers.append(answer)
    
#     return answers

# # Example query
# query = "I can't log into my account"
# answers = retrieval_qa_chain(query)
# # print(answers)
# for answer in answers:
#     print(answer)

["You are an assistant for question-answering tasks. Based on the following document, provide relevant knowledge to answer the query. If the document doesn't contain relevant information, say that you don't know. In the answer only give the relevant knowledge corresponding to the ticket.\n\nDocument:\nSupport Ticket: My account login is not working. I've tried resetting my password twice.\nRelevant Knowledge: Category 1 - Login Issues - Login issues often occur due to incorrect passwords or account lockouts.\n\nQuery:\nI can't log into my account and it's not working. It's not working", "You are an assistant for question-answering tasks. Based on the following document, provide relevant knowledge to answer the query. If the document doesn't contain relevant information, say that you don't know. In the answer only give the relevant knowledge corresponding to the ticket.\n\nDocument:\nSupport Ticket: I can't find the option to change my profile picture.\nRelevant Knowledge: Category 4 - 

In [9]:
# classification_system_prompt = """You are an assistant for classifying support tickets. Based on the following retrieved support tickets and their corresponding knowledge base categories, classify the given query into the most similar category. Only return the category name and number, nothing else.

# Retrieved tickets and categories:
# {retrieved_docs}

# Query to classify: {query}

# Classify the query and respond with only the category name and number. For example, if the query is similar to a login issue, respond with just 'Category 1 - Login Issues'. Only respond from what is mentioned in relevant knowledge and do not generate new text"""

# classification_prompt = PromptTemplate(
#     input_variables=["query", "retrieved_docs"],
#     template=classification_system_prompt
# )

In [10]:
# def classify_ticket(query, retrieved_docs):
#     # Format the retrieved documents
#     formatted_docs = "\n".join(doc.page_content for doc in retrieved_docs)
    
#     # Create a formatted prompt with the query and retrieved documents
#     prompt = classification_prompt.format(query=query, retrieved_docs=formatted_docs)
    
#     # Get the classification from the LLM
#     classification = llm(prompt)
    
#     return classification.strip()

In [ ]:
# classification = classify_ticket(query, retrieved_docs)
# print(classification)

In [14]:
from langchain_nvidia_ai_endpoints import ChatNVIDIA
llm = ChatNVIDIA(model = LLM, temperature=0, max_tokens=500)

In [15]:
classification_system_prompt = """You are an assistant for classifying support tickets. Based on the following retrieved support tickets and their corresponding knowledge base categories, classify the given query into the most similar category. Only return the category name and number, nothing else.

Retrieved tickets and categories:
{retrieved_docs}

Query to classify: {query}

Classify the query and respond with only the category name and number. For example, if the query is similar to a login issue, respond with just 'Category 1 - Login Issues'. Only respond from what is mentioned in relevant knowledge and do not generate new text"""

classification_prompt = PromptTemplate(
    input_variables=["query", "retrieved_docs"],
    template=classification_system_prompt
)

In [42]:
def classify_ticket(query, retrieved_docs):
    # Format the retrieved documents
    formatted_docs = "\n".join(doc.page_content for doc in retrieved_docs)
    
    # Create a formatted prompt with the query and retrieved documents
    prompt = classification_prompt.format(query=query, retrieved_docs=formatted_docs)
    
    # Get the classification from the LLM
    classification = llm(prompt)
    
    return classification.strip()

In [18]:
from langchain_core.messages import HumanMessage, SystemMessage

prompt=query

messages = [
    SystemMessage(content=classification_system_prompt),
    HumanMessage(content=prompt)
]

start = time.time()
classification = llm(messages)
end = time.time()

print(classification)

time = end - start
print(f'\nModel Runtime: {time} Seconds')


content='Based on the retrieved tickets and categories, I would classify the query as:\n\nCategory 5 - Video Playback Issues' response_metadata={'role': 'assistant', 'content': 'Based on the retrieved tickets and categories, I would classify the query as:\n\nCategory 5 - Video Playback Issues', 'token_usage': {'prompt_tokens': 141, 'total_tokens': 163, 'completion_tokens': 22}, 'finish_reason': 'stop', 'model_name': 'meta/llama-3.1-8b-instruct'} id='run-1cb5d172-c103-4fd1-98b9-19d3bd6cbecb-0' role='assistant'

Model Runtime: 0.433307409286499 MilliSeconds


In [17]:
#classification = classify_ticket(query, retrieved_docs)


content='Based on the retrieved tickets and categories, I would classify the query as:\n\nCategory 5 - Video Playback Issues' response_metadata={'role': 'assistant', 'content': 'Based on the retrieved tickets and categories, I would classify the query as:\n\nCategory 5 - Video Playback Issues', 'token_usage': {'prompt_tokens': 141, 'total_tokens': 163, 'completion_tokens': 22}, 'finish_reason': 'stop', 'model_name': 'meta/llama-3.1-8b-instruct'} id='run-b6022f5f-b955-4447-8fc9-d5baa498dd0d-0' role='assistant'
